# Remote

In [1]:
# Set working directory
domain = "fend01"
dir = "exp/ortho_h1-0"
partition = "sbinlab_gpu"

remotedir = f"~/OrthoIDP/{dir}"

print(f"CONTENTS IN {remotedir} AT {domain.upper()}:")
!ssh $domain "cd $remotedir && ls"

print(f"\nCONTENTS IN {remotedir}/results AT {domain.upper()}:")
!ssh $domain "cd $remotedir/results && ls"

CONTENTS IN ~/OrthoIDP/exp/ortho_h1-0 AT FEND01:
data
H1-0_orthodb.json
old.zip
ortho_h1-0.ipynb
ortho_h1-0.json
r0_scan.py
r0_scan.sh
results
submit_r0_scan.sh
submit.sh

CONTENTS IN ~/OrthoIDP/exp/ortho_h1-0/results AT FEND01:
0_AUSLI
100_LEPWE
101_LYNCA
102_MUSPF
103_MYODS
104_MYOLU
105_NEOSC
106_ODORO
107_ODOVR
109_PANPR
10_NEOBR
110_9CHIR
111_PTEAL
112_PTEVA
113_RHIFE
114_RHIFE
115_RHIFE
116_SURSU
117_PIG
118_URSAM
119_URSMA
11_OREAU
120_VICPA
121_VULVU
122_ZALCA
123_CASCN
124_CAVPO
125_CRIGR
126_CRIGR
127_DIPOR
128_FUKDA
12_9CICH
129_ICTTR
130_MESAU
131_MUSCR
132_MOUSE
133_OCTDE
134_OCTDE
135_OCTDE
136_OCTDE
137_OCTDE
138_OCTDE
139_PERMB
13_9TELE
140_RAT
141_CERAT
142_CHLSB
143_COLAP
144_MACFA
145_MACMU
146_MACNE
147_MANLE
148_PAPAN
149_RHIBE
14_ACIRT
150_RHIRO
151_GORGO
152_HUMAN
153_PANPA
154_PANTR
155_PONAB
156_NOMLE
157_PHACI
158_VOMUR
159_AOTNA
15_ACIRT
160_CARSF
161_GEOSA
162_GEOSA
163_LOXAF
164_9AMPH
165_9AMPH
166_ORNAN
167_OTOGA
168_SAIBB
169_TRIMA
16_AMPOC
170_TUPCH
172_

In [2]:
# Check availability
!ssh $domain sinfo -p $partition

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
PARTITION   AVAIL  TIMELIMIT  NODES  STATE NODELIST
sbinlab_gpu    up   infinite      1 drain* node152
sbinlab_gpu    up   infinite      2    mix node[150-151]


In [5]:
# Check queue
!ssh $domain "squeue -u fknudsen"

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          38293963 sbinlab_g idpevo_h fknudsen  R 5-04:34:27      1 node151
        38325169_0 sbinlab_g  initial fknudsen  R    5:33:05      1 node150
        38325169_1 sbinlab_g  initial fknudsen  R    5:33:05      1 node150
        38325169_2 sbinlab_g  initial fknudsen  R    5:33:05      1 node150


In [27]:
# Check submit script
!ssh $domain "cat $remotedir/submit.sh"

#!/bin/bash
#SBATCH --job-name=initial
#SBATCH --partition=sbinlab_gpu
#SBATCH --array=0-5%6
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --gres=gpu:1
#SBATCH -t 24:00:00
#SBATCH -o results/out
#SBATCH -e results/err


# Getting job input file
input_files=($(ls data/*.fasta))
input_file=${input_files[$SLURM_ARRAY_TASK_ID]}

# Displaying job info
echo "[`date`] STARTED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"

# DeiC env settings
source /groups/sbinlab/fpesce/.bashrc
conda activate openmm

# Submitting simulation
python ../../src/simulate_openmm.py -f $input_file

echo "[`date`] FINISHED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"


In [13]:
# Submit
!ssh $domain "cd $remotedir && sbatch submit.sh"

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
/Users/frederikespersenknudsen/Local/OrthoIDP


In [ ]:
# Cancel job
jobid = ""
!ssh $domain scancel $jobid

In [4]:
# Copy results to BINF
!scp -r "$domain:$remotedir/results" $dir

restart.chk                                   100%   87KB   1.0MB/s   00:00    
simulate.log                                  100%  584    18.2KB/s   00:00    
traj.log                                      100% 2375KB   7.7MB/s   00:00    
top.pdb                                       100%   11KB 266.9KB/s   00:00    
traj.dcd                                      100%   44MB  11.6MB/s   00:03    
system.xml                                    100%   26KB 301.9KB/s   00:00    
traj.dcd                                      100%   15MB  10.7MB/s   00:01    
top.pdb                                       100% 3749   200.0KB/s   00:00    
system.xml                                    100% 9458   661.7KB/s   00:00    
simulate.log                                  100%  511    34.2KB/s   00:00    
restart.chk                                   100%   79KB   2.7MB/s   00:00    
traj.log                                      100% 2366KB   8.5MB/s   00:00    
out                                     

In [7]:
!ls $dir

data		  results	  submit_max.sh  submit_test.sh
idpevo_h1-0.json  submit_cons.sh  submit_min.sh


In [6]:
# Copy single specific files to BINF
filename = "r0_scan.*"
!scp "$domain:$remotedir/results/$filename" "$dir/results/$filename"

r0_scan.csv                                   100%  302KB   6.6MB/s   00:00    
r0_scan.err                                   100% 1536   337.0KB/s   00:00    
r0_scan.out                                   100% 9507     2.3MB/s   00:00    
